Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local-advanced.png)

# Register model and deploy locally with advanced usages

This example shows how to deploy a web service in step-by-step fashion:

 1. Register model
 2. Deploy the image as a web service in a local Docker container.
 3. Quickly test changes to your entry script by reloading the local service.
 4. Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [1]:
import os
import re
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, models
from gensim.summarization.textcleaner import split_sentences
import joblib
from sqlalchemy import create_engine
from datasets import Dataset
import copy
import json

from azureml.core import Run, Workspace, Model
from azureml.core.environment import Environment
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.model import InferenceConfig

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


aci-eur-frc-aa-ss-mlw
aci-eur-frc-aa-ss-rg
francecentral
19518d47-0c8b-4829-a602-c5ced78deb3f


### Set runtime environment

In [3]:
from azureml.core.runconfig import RunConfiguration, DockerConfiguration
docker_configuration = DockerConfiguration(use_docker=True)

In [4]:
run = Run.get_context()
keyvault = ws.get_default_keyvault()
rp_env = Environment(name='rp')
#rp_env.docker.enabled = True
#rp_env.docker = docker_configuration
rp_env.python.user_managed_dependencies = True
rp_env.docker.base_image_registry.address = "acieurfrcaassacr.azurecr.io"
rp_env.docker.base_image_registry.username = keyvault.get_secret(name='acieurfrcaassacr-admin-user')
rp_env.docker.base_image_registry.password = keyvault.get_secret(name='acieurfrcaassacr-admin-pwd')
rp_env.inferencing_stack_version = "latest"
rp_env.docker.base_image = "acieurfrcaassacr.azurecr.io/azureml-env-base-research-platform:latest"


In [5]:
inference_config = InferenceConfig(entry_script="inference_search.py",
                                   environment=rp_env)

## Deploy Model as a Local Docker Web Service

*Make sure you have Docker installed and running.*

Note that the service creation can take few minutes.

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

    # PowerShell command to switch to Linux engine
    & 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [6]:
neural_model_name = 'c19gq_ance_msmarco_passage'

In [7]:
model = Model(ws, neural_model_name, version=1)

In [8]:
#model

In [9]:
from azureml.core.webservice import LocalWebservice

In [10]:
# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [11]:
deployment_config

#docker container prune -f

In [12]:
local_service = model.deploy(ws, "test", [model], inference_config, deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry acieurfrcaassacr.azurecr.io
Logging into Docker registry acieurfrcaassacr.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM acieurfrcaassacr.azurecr.io/azureml/azureml_035bb21c11fc36791a93f138d46a324c
 ---> be6a190a2700
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 0ba8c22da2ca
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjE5NTE4ZDQ3LTBjOGItNDgyOS1hNjAyLWM1Y2VkNzhkZWIzZiIsInJlc291cmNlR3JvdXBOYW1lIjoiYWNpLWV1ci1mcmMtYWEtc3MtcmciLCJhY2NvdW50TmFtZSI6ImFjaS1ldXItZnJjLWFhLXNzLW1sdyIsIndvcmtzcGFjZUlkIjoiMzA0ODQ5MmEtZWQ4MC00NGU1LThhZDAtNzc0YjhiNTIzMDY5In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 0e180ff72335
 ---> 655307f70090
Step 4/5 : RUN mv '/var/azureml-app/tmpgv3rf6t6.py' /var/azureml-app/main.py
 ---> Running in 53f178cbf720
 ---> 345a4b7a4de6
Ste

In [13]:
print('Local service port: {}'.format(local_service.port))

Local service port: 6789


## Check Status and Get Container Logs


In [38]:
print(local_service.get_logs())

2021-06-04T06:41:05,881545987+00:00 - gunicorn/run 
2021-06-04T06:41:05,881622188+00:00 - iot-server/run 
2021-06-04T06:41:05,881627378+00:00 - rsyslog/run 
2021-06-04T06:41:05,882324661+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-06-04T06:41:05,924562745+00:00 - iot-server/finish 1 0
2021-06-04T06:41:05,925354179+00:00 - Exit code 1 is normal. Not restarting iot-server.
File not found: /var/azureml-app/.
Starting HTTP server
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 75
SPARK_HOME not set. Skipping PySpark Initialization.
Loading faiss with AVX2 support.
Successfully loaded faiss with AVX2 support.
Initializing logger
2021-06-04 06:41:08,205 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-06-04 06:41:08,205 | root | INFO | Starting up r

In [39]:
local_service.reload()

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.


## Update Service

If you want to change your model(s), Conda dependencies, or deployment configuration, call `update()` to rebuild the Docker image.

```python

local_service.update(models=[SomeOtherModelObject],
                     deployment_config=local_config,
                     inference_config=inference_config)
```

## Delete Service

In [18]:
local_service.delete()

Container (name:thirsty_nobel, id:3e352b72ecbe41e18c3b68a26ba50abfe580e6cf0776489a0942bdbde474765f) cannot be killed.
Container has been successfully cleaned up.
